## Modelos de Machine Learning y Deep Learning

## 1.Algoritmo de LSTM

In [ ]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import numpy as np

def escalar_por_variables_especificas(dataframe, variables_a_escalar, medias, desviaciones_estandar):
  """
  Escala solo las variables especificadas en un dataframe de forma independiente.

  Args:
    dataframe: El dataframe que se va a escalar.
    variables_a_escalar: Una lista que contiene los nombres de las variables que se van a escalar.
    medias: Un diccionario que contiene la media de cada variable.
    desviaciones_estandar: Un diccionario que contiene la desviación estándar de cada variable.

  Returns:
    El dataframe escalado.
  """
  dataframe_escalado = dataframe.copy()
  for columna in dataframe.columns:
    if columna in variables_a_escalar:
      dataframe_escalado[columna] = (dataframe[columna] - medias[columna]) / desviaciones_estandar[columna]
  return dataframe_escalado

In [ ]:
def add_lags(df, num_pasos_futuro=1):  #unistep >> 1 paso al futuro
    # Crear un lag (retroceso) de 1 en la columna 'Fuel Consumption Rate(t)'
    #df['Fuel Consumption Rate(lag1)'] = df['Fuel Consumption Rate'].shift(1)  # Un lag hacia atrás

    # Crear pasos hacia adelante según el número de pasos futuros deseado
    for i in range(1, num_pasos_futuro + 1):
        df[f'Fuel Consumption Rate(t+{i})'] = df['Fuel Consumption Rate'].shift(-i)  # Shift hacia adelante

    # Eliminar filas con NaN en las nuevas columnas de pasos futuros
    df.dropna(inplace=True)
    
    return df

In [ ]:
#Funcion para generar las particiones, siguiendo la secuencia de las series temporales
def train_val_test_split(serie, tr_size =0.8, vl_size=0.1, ts_size=0.1):
    # Definir nunmero de datos en cada subserie
    N = serie.shape[0]   #Serie seria el df, con las columnas que intervendran en el Modelo
    Ntrain = int(tr_size*N)  # Numero de datos de entrenamiento
    Nval = int(vl_size*N)
    Ntest = int(ts_size*N)

    #Realizar la particion
    train = serie[0:Ntrain]
    val = serie[Ntrain:Nval+Ntrain]
    test = serie[Nval+Ntrain:]

    return train, val, test

In [ ]:
#En Multivariado, poner la variable a predecir, en la columna final
c_variables_lstm = ['Intake Manifold #2 Pressure',
    # 'Air Filter #3 Restriction',
    # 'Air Filter #2 Restriction ',
    # 'Air Filter #1 Restriction ',
    # 'Air Filter #4 Restriction ',
    # 'Percent Fuel Position',
    'Roll',
    #'Engine Speed','Left Rear Parking Brake Oil Pressure',
    'Pitch','Fuel Consumption Rate']


datos_df_lstm = datos[c_variables_lstm]  # Modelo Multivariado
datos_df_lstm.head()

In [ ]:
# Seleccionar todas las columnas excepto la variable objetivo
datos_x_escalar_lstm = datos_df_lstm.drop(columns=['Fuel Consumption Rate'])

variables_a_escalar_lstm = datos_x_escalar_lstm.columns

# Calcular medias y desviaciones estándar para todas las variables
medias_lstm = datos_x_escalar_lstm.mean()
desviaciones_estandar_lstm = datos_x_escalar_lstm.std()

# Escalar solo las variables especificadas en el dataframe train
df_escalado_lstm = escalar_por_variables_especificas(datos_x_escalar_lstm, variables_a_escalar_lstm, medias_lstm, desviaciones_estandar_lstm)

In [ ]:
# Identificamos el target del data original
target_lstm = datos_df_lstm['Fuel Consumption Rate']

#Agregamos la target a nuestro datos escalados previamente
df_escalado_lstm['Fuel Consumption Rate'] = target_lstm

In [ ]:
df_escalado_lstm.head()

In [ ]:
import pandas as pd

#4.1 Eliminando columnas especificas que no aportan informacion ( # errors='ignore':ignore cualquier error si alguna de las columnas especificadas no se encuentra en el DataFrame.)
# datos_escalados_lstm = datos_escalados_lstm.drop(['Fuel Consumption Rate(lag1)'], axis=1, errors='ignore')

datos_escalados_lstm = df_escalado_lstm.copy()

# Verifica el resultado
# Llamar a la función para agregar el lag y obtener el nuevo DataFrame
datos_escalados_lstm = add_lags(datos_escalados_lstm)
datos_escalados_lstm.head()

In [ ]:
#### Renombrar las columnas del df
# Renombrar columnas específicas

datos_escalados_lstm = datos_escalados_lstm.rename(columns={
    'Fuel Consumption Rate': 'Fuel Consumption Rate(lag1)',
    'Fuel Consumption Rate(t+1)': 'Fuel Consumption Rate(target)'
})

In [ ]:
datos_escalados_lstm.head()

In [ ]:
# Lista de características (features) que usaremos para X
FEATURES = datos_escalados_lstm.columns.drop('Fuel Consumption Rate(target)')

# Columna objetivo
TARGET = 'Fuel Consumption Rate(target)'

# Particionar los datos en entrenamiento, validación y prueba
train_lstm, val_lstm, test_lstm = train_val_test_split(datos_escalados_lstm)

# Separar las características (X) y la etiqueta (y) en cada partición

# Conjunto de entrenamiento
X_train_lstm = train_lstm[FEATURES]
y_train_lstm = train_lstm[TARGET]

# Conjunto de validación
X_val_lstm = val_lstm[FEATURES]
y_val_lstm = val_lstm[TARGET]

# Conjunto de prueba
X_test_lstm = test_lstm[FEATURES]
y_test_lstm = test_lstm[TARGET]

# Visualizar los tamaños de los conjuntos
print(f"Tamaño de X_train: {X_train_lstm.shape}, y_train: {y_train_lstm.shape}")
print(f"Tamaño de X_val: {X_val_lstm.shape}, y_val: {y_val_lstm.shape}")
print(f"Tamaño de X_test: {X_test_lstm.shape}, y_test: {y_test_lstm.shape}")

In [ ]:
X_test_lstm.head()

In [ ]:
y_test_lstm.head()

In [ ]:
import numpy as np
import pandas as pd

# Definir el número de timesteps
timesteps = 2  # Ajusta según tus necesidades

# Función para crear secuencias
def create_sequences(data_x, data_y, timesteps):
    X, y, indices = [], [], []
    for i in range(len(data_x) - timesteps):
        X.append(data_x.iloc[i:i+timesteps].values)
        y.append(data_y.iloc[i + timesteps])
        indices.append(data_y.index[i + timesteps])  # Guardar el índice correspondiente
    return np.array(X), np.array(y), indices

# Crear las secuencias para el conjunto de entrenamiento
X_train_seq_lstm, y_train_seq_lstm, train_indices = create_sequences(X_train_lstm, y_train_lstm, timesteps)

# Crear las secuencias para el conjunto de validación
X_val_seq_lstm, y_val_seq_lstm, val_indices = create_sequences(X_val_lstm, y_val_lstm, timesteps)

# Crear las secuencias para el conjunto de prueba
X_test_seq_lstm, y_test_seq_lstm, test_indices = create_sequences(X_test_lstm, y_test_lstm, timesteps)

# Imprimir tamaños para confirmar
print(f"Tamaño de X_train_seq: {X_train_seq_lstm.shape}, y_train_seq: {y_train_seq_lstm.shape}")
print(f"Tamaño de X_val_seq: {X_val_seq_lstm.shape}, y_val_seq: {y_val_seq_lstm.shape}")
print(f"Tamaño de X_test_seq: {X_test_seq_lstm.shape}, y_test_seq: {y_test_seq_lstm.shape}")

In [ ]:
X_test_seq_lstm[:5]

In [ ]:
y_test_seq_lstm[:5]

In [ ]:
test_indices

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

# Fijar valores de los parámetros para asegurar la reproducibilidad
tf.random.set_seed(123)
tf.config.experimental.enable_op_determinism()

# Definir el modelo
N_UNITS = 50  # Tamaño del estado oculto de la celda de Memoria de LSTM
INPUT_SHAPE = (X_train_seq_lstm.shape[1], X_train_seq_lstm.shape[2])  # (timesteps)pasos atrás x n features

modelo_lstm = Sequential()
modelo_lstm.add(LSTM(N_UNITS, input_shape=INPUT_SHAPE))
modelo_lstm.add(Dense(1, activation='linear'))  # Predicción de un solo paso futuro

# Compilar el modelo para un problema de regresión
modelo_lstm.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])  # Ajuste a regresión

# Entrenar el modelo
EPOCHS = 50
BATCH_SIZE = 256
historia = modelo_lstm.fit(
    x = X_train_seq_lstm,
    y = y_train_seq_lstm,
    batch_size = BATCH_SIZE,
    epochs = EPOCHS,
    validation_data = (X_val_seq_lstm, y_val_seq_lstm),
    verbose=2
)

In [ ]:
# Visualización de los resultados
import matplotlib.pyplot as plt

plt.plot(historia.history['loss'], label='train')
plt.plot(historia.history['val_loss'], label='validation')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Train and Validation Loss')
plt.legend()
plt.show()

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

# Obtener las predicciones para el conjunto de prueba
y_pred_lstm = modelo_lstm.predict(X_test_seq_lstm)

# Crear una serie de pandas para las predicciones con el índice correspondiente
y_pred_series = pd.Series(y_pred_lstm.flatten(), index=test_indices)

# Crear el gráfico con Plotly
fig = go.Figure()

# Añadir la serie de valores reales
fig.add_trace(go.Scatter(x=test_indices, y=y_test_seq_lstm, mode='lines', name='Valores Reales', line=dict(color='blue')))

# Añadir la serie de valores predichos
fig.add_trace(go.Scatter(x=y_pred_series.index, y=y_pred_series, mode='lines', name='Valores Predichos', line=dict(color='green')))

# Personalizar el gráfico
fig.update_layout(
    title='Serie Temporal: Valores Reales vs Predichos',
    xaxis_title='Fecha',
    yaxis_title='Consumo Combustible (Ltrs/hora)',
    hovermode='x unified'
)

# Mostrar el gráfico
fig.show()